In [9]:
import pandas as pd

paged_data = pd.read_csv('/mydata/q5.csv')
group_keys = paged_data["n_name"].unique()
print("n_name groups:", group_keys)
paged_data

n_name groups: ['CHINA                    ' 'INDIA                    '
 'INDONESIA                ' 'JAPAN                    '
 'VIETNAM                  ']


,n_name,revenue,l_pageid,o_pageid
0,CHINA,13873.3668,12,3
1,CHINA,72690.6180,121,27
2,CHINA,45979.7580,151,35
3,CHINA,89343.6110,224,52
4,CHINA,13281.1560,234,54
...,...,...,...,...
6718281,VIETNAM,50839.8800,115517549,26843200
6718282,VIETNAM,16274.1600,115517571,26843205
6718283,VIETNAM,11401.4673,115517641,26843221
6718284,VIETNAM,42675.8150,115517652,26843224


In [6]:
n_pages = paged_data.groupby("n_name").agg({"l_pageid": "nunique", "o_pageid": "nunique"})
print(n_pages)

                           l_pageid  o_pageid
n_name                                       
CHINA                       1343735   1313622
INDIA                       1338930   1308984
INDONESIA                   1346373   1316042
JAPAN                       1341644   1311506
VIETNAM                     1344659   1314767


In [8]:
n_pages_orders_before_join = 26840487
n_pages_lineitem_before_join = {
    "CHINA                    ": 101666305,
    "INDIA                    ": 101605019,
    "INDONESIA                ": 101673732,
    "JAPAN                    ": 101614053,
    "VIETNAM                  ": 101681279
}

In [33]:
import numpy as np
l_sum = paged_data.groupby(["n_name", "l_pageid"]).agg({"revenue": "sum"})["revenue"]/n_pages_orders_before_join
group_vars_l = {}
for group_key in group_keys:
    group_data = np.concatenate([l_sum[group_key].to_numpy(), np.zeros(int(n_pages_lineitem_before_join[group_key] - len(l_sum[group_key])))])
    group_var = np.var(group_data)
    group_vars_l[group_key] = group_var
    print(group_key, group_var)
max_var = max(group_vars_l.values())
print("Max variance:", max_var)

CHINA                     3.9892726627042553e-08
INDIA                     3.984986485448273e-08
INDONESIA                 4.005517293949803e-08
JAPAN                     3.9920096863205705e-08
VIETNAM                   4.003781607070795e-08
Max variance: 4.005517293949803e-08


In [28]:
o_sum = paged_data.groupby(["n_name", "o_pageid"]).agg({"revenue": "sum"})["revenue"]/max(n_pages_lineitem_before_join.values())
o_data = np.concatenate([o_sum.to_numpy(), np.zeros(int(n_pages_orders_before_join - len(o_sum)))])
o_var = np.var(o_data)
print(o_var)

4.550073596019586e-08


In [31]:
avg = paged_data.groupby("n_name").agg({"revenue": "sum"}) / n_pages_orders_before_join
avgs = {}
for group_key in group_keys:
    avgs[group_key] = avg.loc[group_key]["revenue"]/n_pages_lineitem_before_join[group_key]
    print(group_key, avgs[group_key])
min_avg = min(avgs.values())
print("min_avg", min_avg)

CHINA                     1.9372288491720066e-05
INDIA                     1.9341518251287523e-05
INDONESIA                 1.9433069640054626e-05
JAPAN                     1.938072315397664e-05
VIETNAM                   1.941314952560153e-05
min_avg 1.9341518251287523e-05


In [55]:
import scipy.stats
z = scipy.stats.norm.ppf(0.999)
n = z**2 * (max_var*1.1 + o_var*1.1) / ((min_avg*0.9)**2) / (0.05**2)
print("n", n)

n 1186366.2958330538


In [56]:
sample_rate_o = n / n_pages_orders_before_join
sample_rate_l = n / min(n_pages_lineitem_before_join.values())
print("sample_rate_o", sample_rate_o)
print("sample_rate_l", sample_rate_l)

sample_rate_o 0.04420062481850846
sample_rate_l 0.011676256817914218


In [57]:
from scipy.stats import norm
import math

def _solve_quadratic(a, b, c):
    return (-b - math.sqrt(b**2 - 4*a*c)) / (2*a), (-b + math.sqrt(b**2 - 4*a*c)) / (2*a)

def get_sample_rate(fp: float, sample_size: int, population_size: int):
    z_val = norm.ppf(1-fp)
    p = _solve_quadratic(a=population_size**2 + z_val**2 * population_size,
                        b=-(2*population_size*sample_size + z_val**2 * population_size),
                        c=sample_size**2)[1]
    return p

sample_rate_o = get_sample_rate(0.001, int(n) + 1, n_pages_orders_before_join*0.9)
sample_rate_l = get_sample_rate(0.001, int(n) + 1, min(n_pages_lineitem_before_join.values())*0.9)
print("sample_rate_o", sample_rate_o)
print("sample_rate_l", sample_rate_l)

sample_rate_o 0.0492478856246085
sample_rate_l 0.013010245799012436


In [37]:
gt = paged_data.groupby(["n_name"]).agg({"revenue": "sum"})
gt

,revenue
n_name,
CHINA,5.286258e+10
INDIA,5.274680e+10
INDONESIA,5.303231e+10
JAPAN,5.285842e+10
VIETNAM,5.298188e+10
